### 3. 시계열 예측하기

In [11]:
import numpy as np
from tensorflow import keras

In [12]:
# 사인함수로 시계열 데이터 만들기

def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin(((time - offsets1) * (freq1 * 10 + 10)))  # 사인 곡선1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))  # 사인 곡선2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)    # 잡음
    return series[..., np.newaxis].astype(np.float32)

In [13]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, n_steps], series[9000:, -1]

#### 1) 기준 성능

##### Naive Forcasting (각 시계열의 마지막 값을 그대로 예측하는 것)

In [14]:
# naive forcasting 

y_pred = X_valid[:, -1]
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

0.020365203

##### 완전 연결 네트워크 

In [15]:
# 완전 연결 네트워크 사용하기

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])

In [16]:
model.compile(optimizer='adam', loss='mse')

In [17]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0941
Epoch 2/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0330
Epoch 3/20
219/219 [==============================] - 0s 989us/step - loss: 0.0210
Epoch 4/20
219/219 [==============================] - 0s 784us/step - loss: 0.0159
Epoch 5/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0134A: 0s - loss: 0.01
Epoch 6/20
219/219 [==============================] - 0s 815us/step - loss: 0.0117
Epoch 7/20
219/219 [==============================] - 0s 818us/step - loss: 0.0105
Epoch 8/20
219/219 [==============================] - 0s 856us/step - loss: 0.0094
Epoch 9/20
219/219 [==============================] - 0s 842us/step - loss: 0.0086
Epoch 10/20
219/219 [==============================] - 0s 965us/step - loss: 0.0078
Epoch 11/20
219/219 [==============================] - 0s 1ms/step - loss: 0.0072
Epoch 12/20
219/219 [==============================] - 0s 867us/step - loss

In [18]:
model.evaluate(X_valid, y_valid)

63/63 [==============================] - 0s 947us/step - loss: 0.0049


0.004870355594903231

#### 2) 간단한 RNN 구현하기

In [19]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

In [20]:
model.compile(optimizer='adam', loss='mse')

In [21]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
219/219 [==============================] - 1s 6ms/step - loss: 0.4450
Epoch 2/20
219/219 [==============================] - 1s 6ms/step - loss: 0.3962
Epoch 3/20
219/219 [==============================] - 1s 6ms/step - loss: 0.3518
Epoch 4/20
219/219 [==============================] - 1s 6ms/step - loss: 0.3089
Epoch 5/20
219/219 [==============================] - 1s 6ms/step - loss: 0.2681
Epoch 6/20
219/219 [==============================] - 1s 6ms/step - loss: 0.2303
Epoch 7/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1966
Epoch 8/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1683
Epoch 9/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1457
Epoch 10/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1277
Epoch 11/20
219/219 [==============================] - 1s 6ms/step - loss: 0.1134
Epoch 12/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0980
Epoch 13/20
219/219 [====

In [22]:
model.evaluate(X_valid, y_valid) 

# 완전 연결 층보다 성능이 좋진 않지만 파라미터가 3개만 쓰였음

63/63 [==============================] - 0s 2ms/step - loss: 0.0151


0.015127567574381828

#### 3) 심층 RNN

In [23]:
# SimpleRNN 여러 층으로 쌓기 (활성화 함수 : tanh)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

In [25]:
# 다른 활성화 함수를 사용하기 위해 출력층을 Dense층으로
# 이를 위해 이제는 마지막 순환층이 된 두번째 층에서 return_sequences=True 제거해야

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

#### 4) 여러 타임 스텝 앞 예측

##### (방법1) 예측값을 다음 스텝의 입력값으로 사용

In [26]:
series = generate_time_series(1, n_steps + 10)
X_new, y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new

In [27]:
for step_ahead in range(10):
    y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)
    
Y_pred = X[:, n_steps:]

In [28]:
Y_pred

array([[[-0.23082507],
        [-0.14905705],
        [ 0.0492443 ],
        [ 0.2805521 ],
        [ 0.5279423 ],
        [ 0.6124417 ],
        [ 0.75798327],
        [ 0.82106704],
        [ 0.77922845],
        [ 0.6122835 ]]], dtype=float32)

##### (방법2) RNN을 훈련하여 다음 값 10개를 한 번에 예측
: 시퀀스-투-벡터 모델 (마지막 스텝에서만 다음 값 10개 예측)
<img src='img/15_1.png' width='200'>

In [29]:
series = generate_time_series(10000, n_steps + 10)
X_train, y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

# X에 대해서는 [, , 0] 안 하고 y에 대해서만 해서
# X에 대해서는 2차원 리스트로 만들고 [[1], [2], ..., [n_steps]]
# y에 대해서는 그냥 1차원 리스트로 만듦

In [30]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)    # 10개의 유닛을 가진 출력층
])

In [31]:
model.compile(loss='mse', optimizer='adam')

In [71]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 14s 64ms/step - loss: 0.0392 - val_loss: 0.0080
Epoch 2/20
219/219 [==============================] - 10s 48ms/step - loss: 0.0056 - val_loss: 0.0050
Epoch 3/20
219/219 [==============================] - 10s 47ms/step - loss: 0.0040 - val_loss: 0.0037
Epoch 4/20
219/219 [==============================] - 14s 62ms/step - loss: 0.0036 - val_loss: 0.0033
Epoch 5/20
219/219 [==============================] - 18s 81ms/step - loss: 0.0034 - val_loss: 0.0033
Epoch 6/20
219/219 [==============================] - 13s 60ms/step - loss: 0.0033 - val_loss: 0.0032
Epoch 7/20
219/219 [==============================] - 12s 57ms/step - loss: 0.0032 - val_loss: 0.0032
Epoch 8/20
219/219 [==============================] - 15s 66ms/step - loss: 0.0031 - val_loss: 0.0035
Epoch 9/20
219/219 [==============================] - 19s 88ms/step - loss: 0.0031 - val_loss: 0.0030
Epoch 10/20
219/219 [==============================] - 20s 92ms/step - loss: 0.003

In [72]:
Y_pred = model.predict(X_new)

In [73]:
Y_pred   # MSE 0.008

array([[0.4745298 , 0.44532877, 0.44909075, 0.43029204, 0.47094586,
        0.45882994, 0.4321749 , 0.43762532, 0.46150336, 0.43843722]],
      dtype=float32)

##### (방법3) Sequence-to-Sequence RNN
: 모든 타임 스텝에서 다음 값 10개를 예측하는 모델
<img src='img/15_2.png' width='180'>

In [124]:
# 타깃 시퀀스 만들기 (입력 시퀀스와 동일한 길이의 시퀀스)

Y = np.empty((10000, n_steps, 10))  # 10D벡터의 시퀀스 (0으로 채우기)
for step_ahead in range(1, 10 + 1):
    Y[:, :, step_ahead - 1] = series[:, step_ahead:(step_ahead + n_steps), 0]

y_train = Y[:7000]
y_valid = Y[7000:9000]
y_test = Y[9000:]

In [126]:
print(Y.shape)   # 10차원의 시퀀스 50개
print(X_train.shape)  # 50개의 시퀀스

(10000, 50, 10)
(7000, 50, 1)


In [113]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

# TimeDistributed 층 : 다른 층(Dense 층)을 감싸서 입력 시퀀스의 모든 타임 스텝에
# 적용 (사실 Dense 층이 시퀀스를 입력으로 받을 수 있어 그냥 Dense(10)로 해도 됨)

In [116]:
# 모델 평가하기
# 훈련할 떄는 모든 스텝이 필요하지만, 평가에는 마지막 타임 스텝의 출력만 필요
# -> 마지막 타임 스텝의 출력에 대한 MSE만 계산하는 함수 만듦

def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

optimizer = keras.optimizers.Adam(lr=0.01)
model.compile(loss='mse', optimizer=optimizer, metrics=[last_time_step_mse])

In [117]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 4s 16ms/step - loss: 0.0496 - last_time_step_mse: 0.0388 - val_loss: 0.0428 - val_last_time_step_mse: 0.0326
Epoch 2/20
219/219 [==============================] - 4s 17ms/step - loss: 0.0368 - last_time_step_mse: 0.0248 - val_loss: 0.0336 - val_last_time_step_mse: 0.0224
Epoch 3/20
219/219 [==============================] - 4s 17ms/step - loss: 0.0316 - last_time_step_mse: 0.0196 - val_loss: 0.0287 - val_last_time_step_mse: 0.0153
Epoch 4/20
219/219 [==============================] - 4s 17ms/step - loss: 0.0287 - last_time_step_mse: 0.0168 - val_loss: 0.0265 - val_last_time_step_mse: 0.0139
Epoch 5/20
219/219 [==============================] - 4s 16ms/step - loss: 0.0267 - last_time_step_mse: 0.0150 - val_loss: 0.0254 - val_last_time_step_mse: 0.0136
Epoch 6/20
219/219 [==============================] - 4s 17ms/step - loss: 0.0242 - last_time_step_mse: 0.0122 - val_loss: 0.0234 - val_last_time_step_mse: 0.0113
Epoch 7/20
219/219 [==

In [127]:
model.evaluate(X_test, y_test)   # MSE 0.006

32/32 [==============================] - 0s 4ms/step - loss: 0.0186 - last_time_step_mse: 0.0075


[0.01863916777074337, 0.007499366067349911]

### 4. 긴 시퀀스 다루기
#### 1) 불안정한 그레디언트
- 수렴하는 활성화 함수 사용하기
- 층 정규화
- 드롭아웃

In [32]:
# 층 정규화

class LNSimpleRNNCell(keras.layers.Layer):
    def __init__(self, units, activation='tanh', **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = keras.layers.SimpleRNNCell(units,
                                                         activation=None)
        self.layer_norm = keras.layers.LayerNormalization()
        self.activation = keras.activations.get(activation)
    
    def call(self, inputs, states):
        outputs, new_states = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]  # 두 값을 반환 (은닉상태와 출력)

In [33]:
model = keras.models.Sequential([
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True,
                    input_shape=[None, 1]),
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

# 드롭 아웃은 dropout(입력 드롭아웃 비율)과 recurrent_dropout 옵션(은닉 상태
# 드롭아웃 비율)으로 설정가능

#### 2) 단기 기억 문제 해결하기
- LSTM 셀
- Peephole 연결
- GRU 셀
- 1D 합성곱층 사용하기
- WaveNet

##### LSTM

In [40]:
model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [39]:
# RNN층에 LSTMCell을 매개변수로 지정할 수도 있음 (위와 같은 코드)

model = keras.models.Sequential([
    keras.layers.RNN(keras.layers.LSTMCell(20), return_sequences=True,
                    input_shape=[None, 1]),
    keras.layers.RNN(keras.layers.LSTMCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

##### 1D 합성곱 층을 사용해 시퀀스 처리하기

In [128]:
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding='valid',
                       input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [129]:
model.compile(loss='mse', optimizer=optimizer, metrics=[last_time_step_mse])

In [130]:
history = model.fit(X_train, y_train[:, 3::2], epochs=20,
                    validation_data=(X_valid, y_valid[:, 3::2]))

Epoch 1/20
219/219 [==============================] - 5s 21ms/step - loss: 0.0355 - last_time_step_mse: 0.0280 - val_loss: 0.0190 - val_last_time_step_mse: 0.0086
Epoch 2/20
219/219 [==============================] - 4s 18ms/step - loss: 0.0168 - last_time_step_mse: 0.0066 - val_loss: 0.0152 - val_last_time_step_mse: 0.0056
Epoch 3/20
219/219 [==============================] - 4s 20ms/step - loss: 0.0145 - last_time_step_mse: 0.0050 - val_loss: 0.0137 - val_last_time_step_mse: 0.0043
Epoch 4/20
219/219 [==============================] - 5s 21ms/step - loss: 0.0135 - last_time_step_mse: 0.0044 - val_loss: 0.0140 - val_last_time_step_mse: 0.0049
Epoch 5/20
219/219 [==============================] - 4s 19ms/step - loss: 0.0129 - last_time_step_mse: 0.0040 - val_loss: 0.0128 - val_last_time_step_mse: 0.0042
Epoch 6/20
219/219 [==============================] - 5s 24ms/step - loss: 0.0124 - last_time_step_mse: 0.0037 - val_loss: 0.0127 - val_last_time_step_mse: 0.0042
Epoch 7/20
219/219 [==

##### WAVENET
- 이 네트워크는 층마다 팽창 비율(각 뉴런의 입력이 떨어져 있는 간격)을 두 배로 늘리는 1D 합성곱 층을 쌓음
- 하위층은 단기 패턴을 학습하고 상위층은 장기 패턴을 효율적으로 학습
- 각 층 이전의 팽창 비율과 동일한 개수의 0을 입력 시퀀스 왼쪽에 패딩으로 추가 (causal padding) -> 다음 값을 훔쳐보지 않도록!
<img src='img/15_3.png' width='350'>

In [132]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None, 1]))  # 입력층 
                                                       # for문 안에 넣기엔 복잡
for rate in (1, 2, 4, 8) * 2:  # 1, 2, 4, 8 반복
    model.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding='causal',
                                 activation='relu', dilation_rate=rate))
    
model.add(keras.layers.Conv1D(filters=10, kernel_size=1))

In [133]:
model.compile(loss='mse', optimizer='adam', metrics=[last_time_step_mse])

In [134]:
history = model.fit(X_train, y_train, epochs=20,
                   validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0657 - last_time_step_mse: 0.0543 - val_loss: 0.0360 - val_last_time_step_mse: 0.0228
Epoch 2/20
219/219 [==============================] - 2s 10ms/step - loss: 0.0320 - last_time_step_mse: 0.0196 - val_loss: 0.0295 - val_last_time_step_mse: 0.0175
Epoch 3/20
219/219 [==============================] - 2s 10ms/step - loss: 0.0282 - last_time_step_mse: 0.0162 - val_loss: 0.0277 - val_last_time_step_mse: 0.0154
Epoch 4/20
219/219 [==============================] - 2s 10ms/step - loss: 0.0263 - last_time_step_mse: 0.0143 - val_loss: 0.0256 - val_last_time_step_mse: 0.0136
Epoch 5/20
219/219 [==============================] - 2s 10ms/step - loss: 0.0250 - last_time_step_mse: 0.0130 - val_loss: 0.0247 - val_last_time_step_mse: 0.0127
Epoch 6/20
219/219 [==============================] - 2s 10ms/step - loss: 0.0241 - last_time_step_mse: 0.0122 - val_loss: 0.0241 - val_last_time_step_mse: 0.0125
Epoch 7/20
219/219 [==